In [1]:
from datasets import load_dataset, load_metric
# from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from transformers import BertTokenizer, BertForSequenceClassification, EarlyStoppingCallback
#BertForSequenceRegression https://github.com/ceshine/pytorch-pretrained-BERT/blob/master/notebooks/Sequence%20Regression%20Model.ipynb
from transformers import Trainer, TrainingArguments, EvalPrediction
import pandas as pd
import torch
import wandb
torch.cuda.empty_cache()

In [2]:
train_data = pd.read_csv('data/si630w22-hw3-train.csv')
dev_data = pd.read_csv('data/si630w22-hw3-dev.csv')
# test_data = pd.read_csv('data\si630w22-hw3-test.public.csv')
q_and_a_data = pd.read_csv('data/si630w22-hw3-data.csv')

In [71]:
#combine the the different ratings
train_df = train_data.groupby('id').mean().round()
train_df = train_df.reset_index()
dev_df = dev_data.groupby('id').mean().round()
dev_df = dev_df.reset_index()

In [105]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

train_df.rename(columns={'id':'question_id'}, inplace = True)
dev_df.rename(columns={'id':'question_id'}, inplace = True)
train_df.rename(columns={'rating':'labels'}, inplace = True)
dev_df.rename(columns={'rating':'labels'}, inplace = True)

q_and_a_data['text'] = q_and_a_data.question_text + '[SEP]' + q_and_a_data.reply_text

merged_train_data = pd.merge(train_df,q_and_a_data[['text','question_id']], on='question_id', how='left')
merged_train_data.dropna(subset=['labels'], inplace = True)
merged_train_data.drop(columns=['question_id'], inplace = True)
merged_train_data.to_csv('merged_train_data.csv', index=False)

# One hot encode labels
merged_train_labels_transformed = ohe.fit_transform(merged_train_data.labels.to_numpy().reshape(-1,1))
train_labels_transformed = pd.DataFrame.sparse.from_spmatrix(merged_train_labels_transformed, index=None, columns=[1, 2, 3, 4, 5])
train_transformed = pd.concat([merged_train_data, train_labels_transformed], axis=1)
#train_transformed['ohe_labels'] = train_transformed[[1, 2, 3, 4, 5]].apply(lambda x: list([x['1'],x['2'],x['3'],x['4'],x['5']]),axis=1)  
#train_transformed.to_csv('train_data_transformed.csv', index=False)

merged_dev_data = pd.merge(dev_df, q_and_a_data[['text','question_id']], on='question_id', how='left')
merged_dev_data.dropna(subset=['labels'], inplace = True)
merged_dev_data.drop(columns=['question_id'], inplace = True)
merged_dev_data.to_csv('merged_dev_data.csv', index=False)

# one hot encode labels
merged_dev_labels_transformed = ohe.transform(merged_dev_data.labels.to_numpy().reshape(-1,1))
dev_labels_transformed = pd.DataFrame.sparse.from_spmatrix(merged_dev_labels_transformed, index=None, columns=[1, 2, 3, 4, 5])
dev_transformed = pd.concat([merged_dev_data, dev_labels_transformed], axis=1)
dev_transformed.to_csv('dev_data_transformed.csv', index=False)

# merged_test_data = pd.merge(q_and_a_data, test_data, on='question_id')
# merged_test_data.to_csv('data\merged_test_data.csv', index=False)

TypeError: SparseArray does not support item assignment via setitem

In [109]:
train_transformed.head()
# take numpy array output from one hot encoder and add it as a dictionary to dataset object
# from datasets import Dataset
# my_dict = {"a": [1, 2, 3]}
# dataset = Dataset.from_dict(my_dict)
# https://huggingface.co/docs/datasets/loading

,labels,text,1,2,3,4,5
0,5.0,"Is there someone you turned down in the past, ...",0.0,0.0,0.0,0.0,1.0
1,4.0,"What is, in your opinion, the saddest villain ...",0.0,0.0,0.0,1.0,0.0
2,4.0,ELI5: How do we still not know how eels reprod...,0.0,0.0,0.0,1.0,0.0
3,4.0,ELI5: Why can’t freshwater fish live in saltwa...,0.0,0.0,0.0,1.0,0.0
4,4.0,What's something nice you like to do just to b...,0.0,0.0,0.0,1.0,0.0


In [104]:
data_files = {"train": "train_data_transformed.csv",
              "dev": "dev_data_transformed.csv",}
#               "test": "data\merged_test_data.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Using custom data configuration default-d50e2502e3de8197


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\sryanlee\.cache\huggingface\datasets\csv\default-d50e2502e3de8197\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', '1', '2', '3', '4', '5'],
        num_rows: 3779
    })
    dev: Dataset({
        features: ['labels', 'text', '1', '2', '3', '4', '5'],
        num_rows: 811
    })
})

In [74]:
# dataset['train']['text']

In [75]:
model = BertForSequenceClassification.from_pretrained("microsoft/MiniLM-L12-H384-uncased", 
                                                      problem_type="multi_label_classification",
                                                     num_labels=5)
#https://discuss.huggingface.co/t/mismatched-target-and-input-size-for-bce-using-multi-label-classification/8706

# tokenizer = BertTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
tokenizer = BertTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased", padding = True, truncation=True ,max_length =512)

loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at C:\Users\sryanlee/.cache\huggingface\transformers\ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_clas

In [76]:
tokenized_dataset= dataset.map(lambda x: tokenizer(x['text'],padding = 'max_length', max_length =512))

  0%|          | 0/3779 [00:00<?, ?ex/s]

  0%|          | 0/811 [00:00<?, ?ex/s]

In [77]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3779
    })
    dev: Dataset({
        features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 811
    })
})

In [78]:
tokenized_dataset['train']

Dataset({
    features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3779
})

In [85]:
tokenized_dataset['train']['labels']

tensor([5., 4., 4.,  ..., 4., 2., 3.])

In [79]:
# tokenized_dataset
len(tokenized_dataset['train']['input_ids'][2])
# tokenized_dataset['train'][0]

512

In [80]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask','labels'])

dataloader = torch.utils.data.DataLoader(tokenized_dataset['train'], batch_size=8)
# next(iter(dataloader))
# tokenized_dataset['train'][0]

In [65]:
# training_args = TrainingArguments("test-trainer")
training_args = TrainingArguments(
    output_dir = 'BERTSeq',
    num_train_epochs = 1,
    
#     do_train = True,
#     do_eval = True,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    learning_rate=1e-4,
#     logging_strategy = 'epoch',
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
#     warmup_steps = 250,
#     weight_decay = 0.01,
    seed =0,
#     metric_for_best_model = 'eval_loss',
    load_best_model_at_end = True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [84]:
tokenized_dataset['train']['labels'][0]

tensor(5.)

In [81]:
# more compute metrics
# https://discuss.huggingface.co/t/why-do-i-get-this-error-running-tokenizer/780
# transformers.EvalPrediction https://huggingface.co/docs/transformers/internal/trainer_utils

# from https://theaisummer.com/hugging-face-vit/
# from datasets import load_metric
# metric = load_metric("accuracy")
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     return metric.compute(predictions=predictions, references=labels)

# from https://huggingface.co/transformers/v3.0.2/training.html#trainer
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# METRICS FOR REGRESSION
# from sklearn.metrics import mean_squared_error
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     mse = mean_squared_error(labels, preds)
#     return {
#         'mse': mse
#     }

In [82]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    compute_metrics=compute_metrics,
    #compute_mse(tokenized_dataset['dev']['labels'],tokenized_dataset['dev']['labels']),
#     data_collator=data_collator,
#     tokenizer=tokenizer
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [83]:
trainer.train()

# ValueError thread, item 8/13 https://discuss.pytorch.org/t/target-size-torch-size-10-must-be-the-same-as-input-size-torch-size-2/72354/10
# ValueError thread, define num_labels in model https://discuss.huggingface.co/t/mismatched-target-and-input-size-for-bce-using-multi-label-classification/8706

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\users\sryanlee\documents\projects\si630\si630_nlp\hw3_annotation_langmodel\hw3_venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3779
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 945
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


ValueError: Target size (torch.Size([4])) must be the same as input size (torch.Size([4, 5]))

In [85]:
ii = trainer.train_dataset[1]
# model(**ii, labels =torch.tensor([1]).unsqueeze(0) )
ii

{'labels': tensor(3.7500),
 'input_ids': tensor([  101,  2054,  2003,  1010,  1999,  2115,  5448,  1010,  1996,  6517,
          6155,  2102, 12700, 10457,  7062,  2412,  1029,   102,  2026,  2158,
          2852, 17655, 20160, 18940, 11624, 14503,  3215,  2001,  2141,  2302,
          2010,  3008,  1010,  5102,  2004,  1037, 25781,  2008,  2288,  7383,
          1010,  1998,  2467, 12386,  2000,  1037, 20228,  4017, 22571,  2271,
          2008,  2002,  2071,  4089,  3786,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            

In [86]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

"[CLS] is there someone you turned down in the past, and now regret not going out with? why? [SEP] idk if this counts but my when i was younger my crush asked me to hang out with her like just two of us, and i kinda just ghosted her for the day because i was too coward to be with her without anyone else around because i'm an huge introvert and i'm shy af also my social skills are really bad... i honestly regret that a lot but at least next time i won't do the same mistake. hopefully... [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [133]:
pd.options.display.max_colwidth = 500
merged_train_data.text.head(1)

0    Is there someone you turned down in the past, and now regret not going out with? Why?[SEP]Idk if this counts but my when I was younger my crush asked me to hang out with her like just two of us, and I kinda just ghosted her for the day because I was too coward to be with her without anyone else around because I'm an huge introvert and I'm shy af also my social skills are really bad... I honestly regret that a lot but at least next time I won't do the same mistake. Hopefully...
Name: text, dtype: object

In [217]:
trainer.train_dataset[0].keys()

dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])

In [134]:
# tokenizer.decode(trainer.train_dataset[1]["token_type_ids"])

In [52]:
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)
# model(**inputs)
# type(inputs), inputs

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [85]:
labels

tensor([[1]])

In [116]:
# x =tokenized_dataset['train'][1]
type(tokenized_dataset['train'][1]), x
for (k, v) in x.items():
    print(k,v)
tuple(x[k] for k in x.keys())

input_ids tensor([  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
         2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
         2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
         2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
         2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
         2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
         1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
          102])
token_type_ids tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
attention_mask tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,


(tensor([  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
          2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
          2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
          2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
          2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
          2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
          1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
           102]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1